 From the previous notebook, "Phrase Sampling (Part 1)" we assemble a collection of functions.

main
    select docs for corpus from a list
    execute the pull
    execute each step in the preprocessing

In [1]:
from pathlib import Path
import re, os, math
from operator import itemgetter 

In [2]:
def loadDocs(author1, *authors2):
    #load a selection of texts by selected authors
    auths = [author1]
    re1 = re.compile('(\w+)')
    for other_author in authors2:
        a1 = str(other_author)
        match = re1.search(a1)
        if match:
            auths.append(match.group())
    docs = {}
    for author in auths:
        print(author)
        data_folder = Path("data/" + author)
        for file in data_folder.iterdir():
            if str(file).endswith(".txt"):
                file_to_open = file
                o = open(file_to_open, 'r')
                documentName = file
                document = list(o)
                docs[documentName] = document
    return docs

In [3]:
def trim_headers(first_document):
    # Determine whether a Project Gutenberg Text
    first_header_index = 0
    second_header_index = 0
    footer_index = 0
    if any("GUTENBERG" in s for s in first_document):
        for first_header_index in range( len(first_document) ):
            if ( ( first_document[first_header_index].find('*END*THE SMALL PRINT!') ) != -1 ) :
                break
            else:
                for first_header_index in range( len(first_document) ):
                    if ( ( first_document[first_header_index].find('START OF THIS PROJECT GUTENBERG') ) != -1 ) :
                        break        
        second_document = list(first_document[first_header_index + 1 :])
        for second_header_index in range( len(second_document) ):
            if ( ( second_document[second_header_index].find('www.gutenberg.org') ) != -1 ) :
                break            
        for footer_index in range( len(first_document) ):
            if ( ( first_document[footer_index].find('End of Project') ) != -1 ) :
                break
            else:
                for footer_index in range( len(first_document) ):
                    if ( ( first_document[footer_index].find('End of the Project') ) != -1 ) :
                        break    
        script = list()
        if (second_header_index < (first_header_index + 100)):
            manuscript = list(first_document[first_header_index +1 + second_header_index +1 : footer_index-1])
        else:
            manuscript = list(first_document[first_header_index +1 : footer_index-1])
    else:
        manuscript = first_document
    return manuscript

In [4]:
def collectLines(script):
    #Compile a list of speakers
    r = re.compile("[A-Z0-9][A-Z0-9]+")
    speakers = []
    for line in script:
        mtch = r.match(line)
        if mtch:
            speakers.append(mtch.group())
    #Omit speakers from the list of text
    s = re.compile(r"\b[A-Z{3}\.]+\b")
    spoken = list(filter(lambda i: not s.search(i), script))
    return speakers, spoken

In [5]:
def sentencer(spoken):
    #Concatenate lines into list entries for future sentence splitting
    newLines = []
    singleLine = ''
    singleLines = []

    #Remove all line returns(ok)
    for j in range(0, len(spoken)):
        spoken[j] = spoken[j].replace('\n', '')

    #Split 5 lines at a time into new list
    for k in range( 0, len(spoken), 3):
        newLines = []
        for line in range( 0, 3 ):
            try:
                newLines.append(' '+spoken[line+k])
            except:
                #print("Index Error at", k, line)
                break
        #Join 5-line groups into one line and append to a list
        singleLine = ''.join(newLines)
        singleLines.append(singleLine)
    
    #Create list of sentences
    sentences = []
    for m in range(0, len(singleLines)):
        mtch = re.findall("[A-Z][^\.!?]*[\.!?]", singleLines[m], re.M|re.I)
        if mtch:
            sentences.append(mtch)
    return sentences

In [6]:
def readStopList():
    #Clean the stopword list
    stoplist = []
    clean_line = []
    data_folder = Path("data/")
    file_to_open = data_folder / "snowball_stop.txt"
    f = open(file_to_open, 'r')
    full_stop = list(f)

    for n in range( 0, len(full_stop), 1 ):
        clean_line = full_stop[n].split('|')
        stoplist.append(clean_line[0])

    for p in range(len(stoplist)):
        stoplist[p] = stoplist[p].replace('\n', '')

    #print(stoplist)
    return stoplist

In [7]:
def collectPhrases(sentences, stoplist):
    # Create list of phrases using stopwords
    phrases = []
    candidate_phrases = []

    for q in range(len(sentences)):
        for r in sentences[q]:
            words = re.split("\\s+", r)
            previous_stop = False

            # Examine each word to determine if it is a phrase boundary marker or part of a phrase or alone
            for w in words:

                if w in stoplist and not previous_stop:
                    # phrase boundary encountered, so put a hard indicator
                    candidate_phrases.append(";")
                    previous_stop = True
                elif w not in stoplist and len(w) > 3:
                    # keep adding words to list until a phrase boundary is detected
                    candidate_phrases.append(w.strip())
                    previous_stop = False

        # Create a list of candidate phrases without boundary demarcation
        phrases = re.split(";+", ' '.join(candidate_phrases))

    # Clean up phrases    
    re2 = re.compile('[^\.!?,"(){}\*:]*[\.!?,"(){}\*:]')
    for s in range(len(phrases)):
        phrases[s] = re.sub(re2, '', phrases[s])
        phrases[s] = phrases[s].strip(' ')
        phrases[s] = phrases[s].replace(' ', '_')
        phrases[s] = phrases[s].replace('__', '_')
        phrases[s] = phrases[s].strip('_')

    for s in range(len(phrases)):
        try:
            phrases.remove('')
            phrases.remove(' ')
            phrases.remove('/n')
        except:
            pass

    for t in range(len(phrases)):
        print(phrases[t])
    
    return phrases

In [8]:
def phraseFreq(phrases):
    # Phrase frequency count
    wordfreq = []
    for u in range(len(phrases)):
        utterance = phrases[u]
        uttcnt = 0
        uttcnt = phrases.count(utterance)
        if uttcnt > 1:
            wordfreq.append(uttcnt)

    zipped = list(zip(phrases, wordfreq))
    sortzip = sorted(zipped, key=itemgetter(1), reverse=True)

    for v in range(len(sortzip)):
        print(sortzip[v])
        
    return sortzip

In [9]:
def wordFreq(sentences):
    #Establish wordList and BREAK OUT THE FUNCTION TO WORDS
    wordList = []
    for u in range(len(sentences)):
        for v in sentences[u]:
            words = re.split("\\s+", v)
            wordList.extend(words)

    #Establish wordDict
    wordDict = {}
    for w in range(len(wordList)):
        newWord = wordList[w]
        newWord = newWord.lower()
        newWord = newWord.replace('.', '')
        wordDict[w] = newWord
    #print(wordDict)

    #Perform word counts on dict
    countDict = {}
    for x in range(len(wordDict)):
        term = wordDict[x]
        count = 1
        for y in range(len(wordDict)):
            try:
                if wordDict[y].find(term) > 0:
                    count += 1
            except:
                pass
            countDict[term] = count

    #for k, v in countDict.items():
        #print(k, v)
    return countDict

In [10]:
def computeIDF(docList):
    # Calculates the weight of rare words across all docs
    import math
    idfDict = {}
    N = len(docList)
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1

    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))

    return idfDict

In [11]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [ ]:
#Functional Main
document = loadDocs('murray_kempton','poe')
for d in document:
    trimmed_doc = trim_headers(document[d])
    the_wurd, of_the_Web = collectLines(trimmed_doc)
    have_a_nice_day = sentencer(of_the_Web)
    stoplist = readStopList()
    google_it_up = collectPhrases(have_a_nice_day, stoplist)
    phraseFreq(google_it_up)
    wordFreq(have_a_nice_day)
    

murray_kempton
poe
full_quarter
Yorkers
color_sampled
1990_poll_believed
their
government_“deliberately_makes_sure
drugs_easily_available
poor_black_neighborhoods
order
This_impression
willful_official_malice_disturbing_enough
registered
contagions_seem_likely
swell
widening_spread
reports
Central_Intelligence_Agency_stimulated
crack_epidemic
inner_cities
flooding_them
cocaine
using
profits
fund_arms
fantasy_leaps
customary_function
distraction
Theories
conspiracy_articles
belief
facts
public_policy_articles
disbelief
whose_comforts
views
instance
absolute_harmony
government
pays_without_worrying_enough
government_fairly_acquitted
damaging
poor
witting_stimulations
That
actual_damage_government_does
poor_taking_away
child’s_entitlement
welfare_allotment
guaranteeing_Medicare
seventieth_birthday_present
That
conspiracy
utmost
Golden
claims
find_evidence”
turns
blessed
close
brother
highly_placed
contra_command
peer
This_article_available
Please_choose
options
access
Purchase
Online_Edit

hoped
lighten
brig
cutting_away
were
signs
dragged
bodies
deck
next_care
having_found_axes
rest
stood
stays
whole_mass
wood
rigging_plunged
clear
without_doing
spite
gain_upon
leak_without
done_much_longer
hands_were_entirely
were_bleeding
position
going_overboard
wreck_took
left
reason
rejoice
escape
received
damage
huge_seas
come
even
windlass_shattered
hardly_possible
minutes
blow
About_four
afternoon_utterly_impossible
stand
violence
blast
vessel
hold_together
midnight_settled
deep
away_lifting
portion
brig_entirely
thumped
descent
concussion
occasioned
hold
being_rigged_timber
succession
others
manner
rudder_being_thus_held
stern-post
swinging_freely
entirely
being_clinched
were_drawn_every
them_completely
scarcely_time
draw_breath
violence
board
sweeping
bursting
filling_every_inch
vessel
deck
This_precaution_alone_saved
still_hope
nature
brig
being_every_chance
gale
blow
night
dark
possibly
horrible_shrieking
confusion
surrounded_useless
rather_were_encircled
towering_ridge
port

impress_upon_Captain
necessity
spoke
tempting
opportunity
solving
great_problem
regard
Antarctic_continent_never_indignation
timid
ill-timed_suggestions
head
effect
inducing
push_secrets
continued
pleasant_weather
sounding-gear
hundred
fifty_fathoms
found
current_setting_toward
pole
rate
mile
wind
caused
degree
even
different_quarters
mind
variation
course
several_large_whales
innumerable_flights
albatross_passed
like
carcass
claws
resembling_coral
body_covered
tail_peaked_like
foot
head_resembled
exception
ears--these_were_flopped_like
ears
teeth_were
brilliant_scarlet
January
group
interior_seemed
circumstance
land_came
anchor
league
strong_ripples
rendered
nearer_approach
among
were_Peters
proceeded
look
opening
reef
appeared
encircle
four_large_canoes
filled
seemed
distinguish
They_continued
least_half
good_opportunity
might_have_been_fifty_feet_long
were
hundred
savages
ordinary_stature
muscular
Their_complexion
thick
body
degree
except
turned
headed
bottoms
canoes_were_full
black

portion
abbey_tenanted
back
surface
body
usual_rigorous_illness_immediately_been
gave_myself
passionate_waking_visions
listened--in_extremity
sound_came_again--it
Amazement_struggled
bosom
profound
succeeded
nerving_myself
task
duty_thus
cheek
throat
perceptible_warmth_pervaded
whole_frame
even
slight_pulsation
body_took_upon_itself
loathsome_peculiarities
tenant
sterner
apparently
irredeemable_death
agony_wore
aspect
struggle
invisible_foe
struggle
hurry
vigorously
thing
enshrouded_advanced_boldly
palpably
middle
connected
demeanor
rushing_stirred_not--but_gazed_upon
There
disorder
living_Rowena
confronted_Could_indeed_Rowena
blue-eyed_Lady_Rowena_Trevanion
might
mouth
breathing_Lady
cheeks--there_were
roses
noon
might_indeed
fair_cheeks
living_Lady
might
atmosphere
huge_masses
long
dishevelled_hair
blacker
raven_wings
slowly_opened
eyes
figure
stood
meaning
met
attaching_herself
influence
habit
either
deeds
entered
unflinching_heart
intricacies
whose_strange_meaning_burned_themselves

('South_Seas', 36)
('told', 36)
('middle', 36)
('reached_ears', 36)
('full_hour', 36)
('unutterable_transports', 36)
('blood_flowing', 36)
('vessel', 36)
('joining', 36)
('merriment_must', 36)
('absolutely_necessary', 36)
('handwriting', 36)
('possibility', 36)
('partial_cargo', 36)
('This_result_occasioned', 36)
('drunk', 36)
('glass', 36)
('being', 36)
('There_every_appearance', 36)
('vessel', 36)
('wind_without_shipping', 36)
('forecastle', 36)
('three', 36)
('board', 36)
('descending_among', 36)
('lighten', 36)
('sweeping', 36)
('time', 36)
('windlass_were_drawn_sufficiently_tight', 36)
('greatest_agony_lest', 36)
('making', 36)
('good_deal', 36)
('entertained', 36)
('disappointed_contain_myself', 36)
('partner', 36)
('shore_wherever', 36)
('sight', 29)
('remain', 29)
('trap', 29)
('torments', 29)
('slight_movement_audible_among', 29)
('side', 29)
('This_consisted', 29)
('conduct', 29)
('opportunity', 29)
('found', 29)
('severe_labour', 29)
('make', 29)
('bowsprit', 29)
('mind', 29

('followed', 3)
('immediately_leaped', 3)
('think', 3)
('drinking', 3)
('have_found_impossible', 3)
('doing', 3)
('oil-casks_piled_nearly', 3)
('remove', 3)
('getting', 3)
('probably_have_been_sacrificed', 3)
('means', 3)
('between-decks', 3)
('forged_letter', 3)
('wounds', 3)
('must', 3)
('allowance', 3)
('nearly', 3)
('hourly_expectation', 3)
('Barnard_having_been', 3)
('great_also_came_across', 3)
('closeness', 3)
('pathway_utterly_blocked', 3)
('possibility', 3)
('resolved', 3)
('absence', 3)
('indeed', 3)
('fail', 3)
('danger', 3)
('encountered', 3)
('friendship', 3)
('faith', 3)
('proceeded', 3)
('sufficient_inducement', 3)
('haste', 3)
('noise_considerable_progress', 3)
('fall', 3)
('stowage', 3)
('communicated', 3)
('went', 3)
('distinguish', 3)
('feebleness', 3)
('great_point', 3)
('whether', 3)
('means', 3)
('tightly', 3)
('found', 3)
('rend', 3)
('interstices_consequent_upon', 3)
('given', 3)
('loose_articles', 3)
('necessity', 3)
('compactly', 3)
('extending_completely_acro